In [11]:
import re
import sys
import time
sys.path.insert(0, './..')

from openai import OpenAI

from utils import load_dataset, calculate_classification_metric, write_classification_metric

In [12]:
dataset = load_dataset('./../dataset/test.csv')

In [13]:
model = 'gpt-4-0125-preview'

In [14]:
prompt = [
    {
        'role': 'system',
        'content': '\
            You will be provided with a string. \
            Distinguish if the string causes SQL injection as a web security specialist.\
            If the string can cause SQL injection, return 1. If not, return 0.\
            Return only result for answer. Do not explain the result.'
    },
    {
        'role': 'user',
        'content': '<string>\' or \'a\' = \'a</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    },
    {
        'role': 'user',
        'content': '<string>Hello.</string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>I am a Japanese<./string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>-3756\' )  union all select 1034,1034,1034--</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    }
]

In [15]:
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

client = OpenAI()

for i, (query, label) in enumerate(dataset[:500]):
    print('i', i)
    prompt.append(
        {
            'role': 'user',
            'content': f'<string>{query}</string>'
        })

    start_time = time.time()
    chat_completion = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.1,
        max_tokens=2
    )
    inference_time.append(time.time() - start_time)

    response = chat_completion.choices[0].message.content
    print('response:', response)
    try:
        result = int(re.search(r'(\d+)', response).group(1))
    except ValueError:
        result = 'N/A'
    except AttributeError:
        result = 'N/A'

    labels.append(int(label))
    raw_queries.append(query)
    preds.append(result)


    print(result, label)
    prompt.pop(9)

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)

i 0
response: 0
0 0
i 1
response: 1
1 1
i 2
response: 1
1 1
i 3
response: 0
0 0
i 4
response: 0
0 0
i 5
response: 0
0 0
i 6
response: 1
1 1
i 7
response: 0
0 0
i 8
response: 1
1 1
i 9
response: 0
0 0
i 10
response: 1
1 1
i 11
response: 0
0 0
i 12
response: 0
0 0
i 13
response: 1
1 0
i 14
response: 0
0 0
i 15
response: 1
1 1
i 16
response: 0
0 0
i 17
response: 1
1 1
i 18
response: 0
0 0
i 19
response: 0
0 0
i 20
response: 0
0 0
i 21
response: 0
0 0
i 22
response: 1
1 1
i 23
response: 0
0 0
i 24
response: 1
1 1
i 25
response: 1
1 1
i 26
response: 0
0 0
i 27
response: 1
1 1
i 28
response: 1
1 1
i 29
response: 1
1 1
i 30
response: 0
0 0
i 31
response: 0
0 0
i 32
response: 0
0 0
i 33
response: 1
1 1
i 34
response: 1
1 1
i 35
response: 0
0 0
i 36
response: 0
0 0
i 37
response: 0
0 0
i 38
response: 1
1 1
i 39
response: 1
1 1
i 40
response: 0
0 0
i 41
response: 0
0 0
i 42
response: 0
0 0
i 43
response: 0
0 0
i 44
response: 0
0 0
i 45
response: 1
1 1
i 46
response: 1
1 0
i 47
response: 1
1 1
i 